## Purpose of this script:

Extract relevant play data fields from multiple tables provided including plays_playlist_game, game_events and NFL_scrapr to create a master play level table with key attributes for modeling and analysis.

In [2]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import json
import s3fs
import re
from datetime import datetime
# import sys
# import csv

## Plays Playlist

In [3]:
#function to download and concatenate all raw data from the plays_playlist_game S3 bucket for a given year
def ppg_consolidator(year):
    fs = s3fs.S3FileSystem()
    s3_path='nyg-hackathon-154843675742/source_file/nfl/plays_playlist_game/year={}/'.format(year)

    # list of all s3 file paths
    games_plays_filenames = fs.ls('s3://{}/'.format(s3_path))
    print('Number of PPG files for {} : '.format(year), len(games_plays_filenames))

    #read in all files and concatenate into one Dataframe
    s3 = boto3.resource('s3')
    plays = []
    for file in games_plays_filenames:
        split_filename = file.split('/',maxsplit=1)
        bucket = split_filename[0]
        datakey = split_filename[1]
        obj= s3.Object(bucket, datakey)
        body = obj.get()['Body'].read().decode('utf-8')
        data = json.loads(body)
        temp = pd.DataFrame(data['plays'])
        plays.append(temp)
    df_ppg = pd.concat(plays).reset_index()
    df_ppg = df_ppg.drop('index', axis=1)
    
    return df_ppg

In [4]:
#function to parse and cleanse plays_playlist_game data
def clean_ppg_table(df):

    #prep sorted dataset for parsing
    dfdown = df.loc[df.down > 0] # Removes kickoffs, possibly more (?)
    dfd_sorted = dfdown.sort_values(by="sequence")
    
    # parses offense and defense columns 
    # note: punts and non-snap plays (and possibly more) don't have the same formation info

    dfd_trans = dfd_sorted.copy()
    parser = lambda xy: [(k.get(xy[1]) if type(k).__name__ == 'dict' else None) for k in dfd_sorted[xy[0]]]

    dfd_trans['oForm'] = parser(('offense','offenseFormation'))
    dfd_trans['oPersonnel'] = parser(('offense','personnel'))
    dfd_trans['dInBox'] = parser(('defense','defendersInTheBox'))
    dfd_trans['dRushers'] = parser(('defense', 'numberOfPassRushers'))
    dfd_trans['dPersonnel'] = parser(('defense', 'personnel'))
    dfd_trans['allPersonnel'] = dfd_trans['oPersonnel'] + ', ' + dfd_trans['dPersonnel']
    
    # 'missing' values show as NaN

    dfd_trans.drop(['offense', 'defense', 'oPersonnel', 'dPersonnel'], axis=1, inplace=True)
    
    #extract number of players in each position
    def research(pos, allpos):
        try:
            return (re.search("\\d(?=\\s"+pos+")", allpos).group(0))
        except:
            return None
    for pos in ['RB', 'TE', 'WR', 'DL', 'LB', 'DB']:
        dfd_trans[pos] = [research(pos, k) for k in dfd_trans['allPersonnel']]
    dfd_trans.drop('allPersonnel', axis=1, inplace=True)
    
    #parse play description field between playtime and description
    dfd_trans['playDesc']=[re.sub('\\(.*?\\)\\s','',x,count=1) for x in dfd_trans['playDescription']]
    dfd_trans['playTime']=[re.search('(?<=\\().*?(?=\\)\\s)',x).group(0) for x in dfd_trans['playDescription']]
    
  
    #filter for only regular season games
    dfd_trans = dfd_trans[dfd_trans['seasonType'] == 'REG'].copy()
    
    #keep relevant columns in final df to export
    ppg_keep = dfd_trans[['gameId', 'playId', 'homeScore', 'visitorScore', 'possessionTeam', 'down',
                          'yardsToGo', 'isRedzonePlay', 'isScoring', 'yardlineNumber', 'yardlineSide', 'oForm',
                          'dInBox', 'dRushers', 'RB', 'TE', 'WR', 'DL', 'LB', 'DB', 'playDesc']]
    
    
    return ppg_keep

## Game Events

In [5]:
#function to download and concatenate all raw data from the game_events S3 bucket for a given year
def ge_consolidator(year):
    fs = s3fs.S3FileSystem()
    s3_path='nyg-hackathon-154843675742/source_file/nfl/game_events/year={}/'.format(year)

    # list of all s3 file paths
    games_events_filenames = fs.ls('s3://{}/'.format(s3_path))
    print('Number of GE files for {} : '.format(year), len(games_events_filenames))

    #read in all files and concatenate into one Dataframe
    s3 = boto3.resource('s3')
    events = []
    for file in games_events_filenames:
        split_filename = file.split('/',maxsplit=1)
        bucket = split_filename[0]
        datakey = split_filename[1]
        obj= s3.Object(bucket, datakey)
        body = obj.get()['Body'].read().decode('utf-8')
        data = json.loads(body)
        temp = pd.DataFrame(data['events'])
        events.append(temp)
    df_ge = pd.concat(events).reset_index()
    df_ge = df_ge.drop('index', axis=1)
    
    return df_ge

In [6]:
#function to parse and cleanse game_events data
def clean_ge_table(df):
    
    #prep sorted dataset for parsing
    dfedown = df.loc[df.down > 0] 
    dfe_sorted = dfedown.sort_values(by="sequence")

    # Drop columns already in PPG (don't drop join cols)
    dfe_sorted.drop(['down', 'isSTPlay', 'isScoring', 'playDescription', 'sequence', 'yardsToGo',
                    'possessionTeamId', 'yardline', 'yardlineNumber', 'yardlineSide'], axis=1, inplace=True)
    
    #extract start and endtime in datetime format
    def datetry(x):
        try:
            return(datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M:%S.%f"))
        except:
            return None

    dfe_sorted['startTime'] = [datetry(x) for x in dfe_sorted.startPlayTime]
    dfe_sorted['endTime'] = [datetry(x) for x in dfe_sorted.endPlayTime]

    dfe_sorted.drop(['startPlayTime', 'endPlayTime'], axis=1, inplace=True)

    #extract snaps
    def getsnap(eventset):
        y = [k['time'] if k['name']=='ball_snap' else None for k in eventset]
        z = list(filter(None.__ne__, y))
        return (z[0] if len(z) > 0 else None)
    
    dfe_sorted['snap'] = [getsnap(x) if type(x) != float else None for x in dfe_sorted.events]
    dfe_snaps = dfe_sorted.loc[pd.notnull(dfe_sorted['snap'])]
    dfe_snaps['snapTime'] = [datetry(x) for x in dfe_snaps['snap']]
    del dfe_snaps['snap']

    # Drop columns we're not keeping for now (including the events, which we can parse and curate later)
    dfe_snaps.drop(['timeOfDayUTC', 'events'], axis=1, inplace=True)
    return dfe_snaps

In [7]:
#funtion to merge playlist_plays and game_events data
def combine_ppg_ge(year):
    ppg_df = clean_ppg_table(ppg_consolidator(2019))
    ge_df = clean_ge_table(ge_consolidator(2019))
    
    df_comb = pd.merge(left=ppg_df, right=ge_df, on=["gameId", "playId"], how="left")
  

    df_final = df_comb[['gameId','playId','homeScore','visitorScore','quarter','gameClock', 
                        'startTime', 'endTime', 'snapTime', 'possessionTeam','isGoalToGo','yardlineNumber','yardlineSide',
                       'absoluteYardlineNumber','isRedzonePlay','isScoring','isPenalty',
                        'oForm','dInBox','RB','TE','WR','DL','LB','DB',
                        'dRushers', 'playDesc' ]]
    
    return df_final

In [8]:
#create clean merged 2019 PPG and GE table
ppg_ge_19 = combine_ppg_ge(2019)

Number of PPG files for 2019 :  332
Number of GE files for 2019 :  333


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### NFLScrapr

In [9]:
# load regular season games
s3b = 'nyg-hackathon-154843675742'
data_key = 'source_file/nfl/nflscrapr_data/season=regular/reg_pbp_2017.csv'
data_location = 's3://{}/{}'.format(s3b, data_key)

scrapr_2017_reg = pd.read_csv(data_location, dtype = str)


data_key = 'source_file/nfl/nflscrapr_data/season=regular/reg_pbp_2018.csv'
data_location = 's3://{}/{}'.format(s3b, data_key)

scrapr_2018_reg = pd.read_csv(data_location, dtype = str)


data_key = 'source_file/nfl/nflscrapr_data/season=regular/reg_pbp_2019.csv'
data_location = 's3://{}/{}'.format(s3b, data_key)

scrapr_2019_reg = pd.read_csv(data_location, dtype = str)

In [12]:
scrapr_include = ['play_id', 'game_id', 'home_team', 'away_team', 'down', 'ydstogo', 'play_type', 'shotgun', 'no_huddle',
 'qb_dropback', 'qb_scramble', 'pass_location', 'air_yards', 'run_location', 'run_gap', 'ep', 'epa', 'wp',
 'home_wp', 'wpa', 'first_down_rush', 'first_down_pass', 'third_down_converted', 'third_down_failed', 
 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'interception', 'fumble_forced', 'tackled_for_loss', 
'fumble_lost', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'fumble', 'complete_pass', 'tackle_for_loss_1_player_id', 
 'tackle_for_loss_1_player_name', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 'qb_hit_2_player_id', 
'qb_hit_2_player_name', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name']

In [13]:
scrapr_2017_reg_short = scrapr_2017_reg[scrapr_include]
scrapr_2018_reg_short = scrapr_2018_reg[scrapr_include]
scrapr_2019_reg_short = scrapr_2019_reg[scrapr_include]

In [14]:
scrapr = pd.concat([scrapr_2017_reg_short, scrapr_2018_reg_short, scrapr_2019_reg_short])

In [16]:
#merge with ppg_ge table
scrapr.rename(columns = {'game_id': 'gameId', 'play_id': 'playId'}, inplace=True)
scrapr['playId'] = scrapr['playId'].astype(int)
scrapr['gameId'] = scrapr['gameId'].astype(int)
merged_df = ppg_ge_19.merge(scrapr, on=['gameId', 'playId'], how='left')

In [19]:
#reorder columns
col_order = ['gameId', 'playId', 'home_team', 'away_team', 'homeScore', 'visitorScore', 'quarter', 'gameClock',
       'startTime', 'endTime', 'snapTime',  'possessionTeam', 'playDesc', 
       'down', 'ydstogo', 'play_type',
       'isGoalToGo', 'yardlineNumber', 'yardlineSide',
       'absoluteYardlineNumber', 'isRedzonePlay', 'isScoring', 'isPenalty',
       'oForm', 'dInBox', 'RB', 'TE', 'WR', 'DL', 'LB', 'DB', 'dRushers',
        'shotgun', 'no_huddle', 'qb_dropback',
       'qb_scramble', 'pass_location', 'air_yards', 'run_location', 'run_gap',
       'ep', 'epa', 'wp', 'home_wp', 'wpa', 'first_down_rush',
       'first_down_pass', 'third_down_converted', 'third_down_failed',
       'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass',
       'interception', 'fumble_forced', 'tackled_for_loss', 'fumble_lost',
       'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'fumble',
       'complete_pass', 'tackle_for_loss_1_player_id',
       'tackle_for_loss_1_player_name', 'qb_hit_1_player_id',
       'qb_hit_1_player_name', 'qb_hit_2_player_id', 'qb_hit_2_player_name',
       'forced_fumble_player_1_player_id',
       'forced_fumble_player_1_player_name']

In [21]:
#export to csv - change version # if code updated and need to recraete dataframe
# merged_df[col_order].to_csv('../data/master_play_table_v1.csv', index=False)